<a href="https://colab.research.google.com/github/KjelleJ/enkla-ai-experiment/blob/main/AIX_7_face_recognition.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

---
#ANSIKTSIGENKÄNNING / FACE RECOGNITION med deepface
https://github.com/serengil/deepface

https://wellsr.com/python/face-recognition-with-python-deepface-library/

https://www.sitepoint.com/keras-face-detection-recognition/

---
Görs i tre steg:
1.   Lokalisera och 'klipp ut' alla ansikten i bilden (detektor)
2.   Skapa en vektor (embedding) för varje ansikte (modell)
3.   Jämför vektorer för att identifiera personer (cosine distance)

Eller mera exakt:
"A modern face recognition pipeline consists of 5 common stages: **detect, align, normalize, represent and verify**. While DeepFace handles all these common stages in the background, you don’t need to acquire in-depth knowledge about all the processes behind it. You can just call its verification, find or analysis function with a single line of code."


In [ ]:
!pip install deepface

In [ ]:
from deepface import DeepFace
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import matplotlib.image as mpimg
import os
import requests
import numpy as np
from matplotlib.patches import Rectangle

In [ ]:
# Funktion: Hämta bild från url, lagra som lokal fil
def get_img_to_file(url, path):
  img_data = requests.get(url).content
  with open(path, 'wb') as handler:
      handler.write(img_data)

---
# ENKELT FALL: 2 BILDER - ÄR DET SAMMA PERSON?
---

In [ ]:
get_img_to_file("https://www.download.gubboit.se/musk.JPG", "musk.JPG")
get_img_to_file("https://www.download.gubboit.se/musk2.JPG", "musk2.JPG")
get_img_to_file("https://gfx-bloggar.aftonbladet-cdn.se/wp-content/blogs.dir/518/files/2015/12/brigitte-1024x800.jpg", "star.jpg")

##Hitta ansikten i en bild

In [ ]:
# extract_faces visar var ett ansikte finns i en bild
result = DeepFace.extract_faces(img_path = "musk.JPG")
result[0]['facial_area']


##Markera ansikten i en bild

In [ ]:
# Funktion: Visa bild med en 'bounding box' per ansikte
def highlight_faces(image_path, faces):
  plt.figure(figsize=(10, 8))
  # display image
  image = plt.imread(image_path)
  plt.imshow(image)

  ax = plt.gca()

  for i in range(len(faces)):
    x = faces[i]['facial_area']['x']
    y = faces[i]['facial_area']['y']
    w = faces[i]['facial_area']['w']
    h = faces[i]['facial_area']['h']

    face_border = Rectangle((x,y), w, h,
                          fill=False, color='red')
    plt.text(x, y, str(i), color="red",
             fontdict={"fontsize":20,"fontweight":'bold',"ha":"left", "va":"baseline"})
    ax.add_patch(face_border)
    #i = i + 1
  plt.axis('off')
  plt.show()

In [ ]:
result = DeepFace.extract_faces(img_path = "musk.JPG")
highlight_faces("musk.JPG", result)

In [ ]:
result = DeepFace.extract_faces(img_path = "musk2.JPG")
highlight_faces("musk2.JPG", result)

In [ ]:
result = DeepFace.extract_faces(img_path = "star.jpg")
highlight_faces("star.jpg", result)

##Jämför ansikten

In [ ]:
result = DeepFace.verify(img1_path = "musk.JPG",
                         img2_path = "musk2.JPG")

In [ ]:
# Om verified=True är det samma person. Då är distance < threshold.
result

In [ ]:
result = DeepFace.verify(img1_path = "musk.JPG",
                         img2_path = "star.jpg")

In [ ]:
result

---
#CHELSEA - 2 LAGBILDER
---

Vilka spelare från den första bilden finns med på den andra bilden?


In [ ]:
# Spara lagbilderna på fil
get_img_to_file("https://www.download.gubboit.se/chelsea1.jpg", "chelsea1.jpg")
get_img_to_file("https://www.download.gubboit.se/chelsea2.jpg", "chelsea2.jpg")

In [ ]:
result1 = DeepFace.represent(img_path = "chelsea1.jpg")
print(len(result1))
result1[0].keys()

In [ ]:
# Alltför dålig detektor, vi måste använda en bättre...
highlight_faces("chelsea1.jpg", result1)

In [ ]:
# Nu med 'retinaface' i stället för 'opencv' och 'Facenet512'i stället för 'VGG-Face'
result1 = DeepFace.represent(img_path = "chelsea1.jpg", detector_backend = 'retinaface', model_name='Facenet512')
result1[0].keys()

In [ ]:
print(len(result1))

In [ ]:
print(result1[0]["facial_area"])

In [ ]:
highlight_faces("chelsea1.jpg", result1)

In [ ]:
result2 = DeepFace.represent(img_path = "chelsea2.jpg", detector_backend = 'retinaface', model_name='Facenet512')


In [ ]:
highlight_faces("chelsea2.jpg", result2)

In [ ]:
# Funktion: Returnera inbäddningar för alla ansikten i en bild
def embeddings(result):
  X = np.zeros((len(result), len(result[0]['embedding'])))
  for i in range(len(result)):
    X[i:] = result[i]['embedding']
  return X

In [ ]:
X1 = embeddings(result1)


In [ ]:
X1.shape

In [ ]:
X2 = embeddings(result2)
X2.shape

In [ ]:
from scipy.spatial import distance

for i in range(len(result1)):
  for j in range(len(result2)):
    cos = distance.cosine(X1[i], X2[j])
    if cos < 0.4: # default-värde
      print(i, j, cos)

In [ ]:
highlight_faces("chelsea1.jpg", result1)
highlight_faces("chelsea2.jpg", result2)

##Laguppställningar
Bild 1: **Kepa**, **Azpilicueta**, Luiz, **Christensen**, **Emerson**, **Kante**, Barkley, **Kovacic**, Hazard, **Pedro**, **Giroud**

Bild 2:: **Kepa**, **Azpilicueta**, **Christensen**, Zouma, **Emerson**, **Kante**, Jorginho, **Kovacic**, **Pedro**, **Giroud**, Pulisic

Det är alltså 8 spelare från bild1 som finns med i bild2. Nr 4, 7 och 9 från bild1 saknas i bild2 enligt prediktionen.

---
#JÄMFÖR BILDER PÅ KÄNDISAR
---
##### Vilka bilder föreställer samma person?
##### Kändisar: Ben Affleck, Elton John, Jerry Seinfeld, Madonna, Mindy Kaling

In [ ]:
# Hämta bilderna. 25 bilder - 5 för varje kändis
!rm -f 5celeb.zip*
!wget download.gubboit.se/5celeb.zip

In [ ]:
!unzip -oq 5celeb.zip

In [ ]:
# 25 kändisbilder 1.jpg - 25.jpg.
# 1-5 Ben, 6-10 Elton, 11-15 Jerry, 16-20 Madonna, 21-25 Mindy
!ls 5celeb

In [ ]:
result = DeepFace.represent(img_path = "5celeb/10.jpg", detector_backend = 'retinaface', model_name='Facenet512')

In [ ]:
result[0]['facial_area']

In [ ]:
# Det här tar en stund...
# För varje kändisbild:
# - detektera ansiktet
# - spara bilden av ansiktet (celeb_faces)
# - spara vektorn som representertar ansiktet (model_scores)
model_scores = []
celeb_faces = []
for i in range(1, 26):
  file = '5celeb/' + str(i) + '.jpg'
  result = DeepFace.represent(img_path = file, detector_backend = 'retinaface', model_name='Facenet512')
  model_scores.append(result[0]['embedding'])
  image = plt.imread(file)
  face = result[0]['facial_area']
  left = face['x']
  top = face['y']
  bottom = top + face['h']
  right = left + face['w']
  face_image = image[top:bottom, left:right]
  celeb_faces.append(face_image)

In [ ]:
# Jämför vektorerna för alla bildpar. Visa bilderna om samma person.
from scipy.spatial import distance
differs = []
same = 0
print("Samma person:")
for idx in range(len(model_scores)):
  for idy in range(idx + 1, len(model_scores)):
    dist = distance.cosine(model_scores[idx], model_scores[idy])
    # 0.55 ger bästa resultat + inga fel...
    if dist < 0.55:
      same = same + 1
      # Printing the IDs of faces
      print(idx+1, idy+1)
      # Displaying each matched pair of faces
      plt.figure(figsize=(4, 4))
      plt.subplot(1, 2, 1)
      plt.axis('off')
      plt.imshow(celeb_faces[idx])
      plt.subplot(1, 2, 2)
      plt.axis('off')
      plt.imshow(celeb_faces[idy])
      plt.show()
    else: # ej samma
      differs.append(str(idx+1) + ':' + str(idy+1))
print("Av 50 bildpar med samma person hittade vi " + str(same))
print("Olika personer:")
print(differs)

In [ ]:
# Alla ansikten

i = 1
while i < 25:
    plt.figure(figsize=(8, 8))
    print(f"{i}-{i+4}")
    for j in range(5):
      plt.subplot(1, 5, j+1)
      plt.axis('off')
      plt.imshow(celeb_faces[i + j - 1])
    plt.show()
    i = i + 5

## Kluster

---

Ett annat sätt att bestämma vilka bilder som föreställer samma person är att gruppera bilderna i kluster. I stället för 'cosine distance' (vinkel mellan två vektorer) används avståndet mellan punkterna motsvarande vektorerna. Tröskelvärden behövs inte. Ingen modell.

In [ ]:
# Varje bild representeras av en vektor
len(model_scores)

In [ ]:
# gör om model_scores till np-arrayer (en matris)
model_scores_np = np.vstack(model_scores)
print(model_scores_np.shape)

In [ ]:
# Vi har 5 kändisar - alltså vill vi ha 5 kluster från KMeans
from sklearn.cluster import KMeans
km = KMeans(n_clusters=5, init='random', n_init=10, max_iter=300, tol=1e-04, random_state=0)
y_km = km.fit_predict(model_scores_np)
# kluster-index för varje bild 1-25
print(y_km)

##Varför får vi mycket bättre resultat med KMeans (100%) än med 'cosine distance' (80%)?
Samma vektorer men...


KMeans skapar kluster baserat på avståndet mellan punkterna (som mosvarar vektorerna). Inget tröskelvärde.

'Cosine distance' använder vinkeln mellan vektorerna samt ett tröskelvärde

Troligtvis är det tröskelvärdet som är problemet - att hitta ett enda passande tröskelvärde. Men jämförelsen är inte helt rättvis: För KMeans utnyttjar vi att vi vet att det är fem kluster.


### Plotta klustren i 2d

In [ ]:
# standardisera - förbättrar resultatet något
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_std = sc.fit_transform(model_scores_np)

In [ ]:
# Minska vektorerna från 512d till 2d med PCA
from sklearn.decomposition import PCA
# init PCA transformer till 2d
pca = PCA(n_components=2)
# dimensionality reduction:
X_pca = pca.fit_transform(X_std)
X_pca.shape

In [ ]:
# Plotta punkterna i 2d
import matplotlib.pyplot as plt
act_part = X_pca[:5]
plt.scatter(act_part[:, 0], act_part[:, 1],
            c='green', marker='x', s=50, label='Ben')
act_part = X_pca[5:10]
plt.scatter(act_part[:, 0], act_part[:, 1],
            c='green', marker='*', edgecolor='green',s=50, label='Elton')
act_part = X_pca[10:15]
plt.scatter(act_part[:, 0], act_part[:, 1],
            c='green', marker='o', edgecolor='green',s=50, label='Jerry')
act_part = X_pca[15:20]
plt.scatter(act_part[:, 0], act_part[:, 1],
            c='red', marker='o', edgecolor='red',s=50, label='Madonna')
act_part = X_pca[20:25]
plt.scatter(act_part[:, 0], act_part[:, 1],
            c='yellow', marker='o', edgecolor='black',s=50, label='Mindy')
plt.legend(scatterpoints=1)
plt.grid()
plt.title("2d representation (från 128d med PCA)")
plt.tight_layout()
plt.show()

---
#Shaderbooth

Prova Shaderbooth på https://shaderbooth.com/

Appen använder webbkameran och 'facial landmarks' för att lägga på olika effekter. Vänta tills appen går igång. Tryck på pilarna för att få olika effekter.